In [1]:
from openai import OpenAI
import time

In [2]:
with open('openai_api_key.py') as script:
    exec(script.readline())

client = OpenAI(
  api_key=openai_api_key,
)

# Sunk Cost Fallacy
### Using OpenAI Playground - Complete API with gpt-3.5-turbo-instruct

Prompt:

*Assume that you have spent `$90/$250` for a ticket to a theater performance. Several weeks later you buy a `$30` ticket to a rock concert. You think you will enjoy the rock concert more than the theater performance. As you are putting your just-purchased rock concert ticket in your wallet, you notice that both events are scheduled for the same evening. The tickets are non-transferable, nor can they be exchanged. You can use only one of the tickets and not the other. Which ticket will you use?*

*Answer: I would use the ticket to the*

---

From:

Soman, D. 2001. The mental accounting of sunk time costs: why time is not like money. Journal of Behavioral Decision Making 14(3): 169–185.

Arkes, H.R., and C. Blumer. 1985. The psychology of sunk cost. Organizational Behavior and Human
Decision Processes 35(1): 124–140.

(both Experiment 1)

---

The OpenAI Complete API Playground allows you to see the probabilities of a token being generated. This allows you to see how likely the model is to choose an option. You can also change the parameters and the amount of sunk cost to see how the probabilities change:

| Parameters |                   | `$30 vs. $90`       |                      | `$30 vs. $250`         |                      |
|---------|------------|----------------------|----------------------|----------------------|----------------------|
|  Temperature | Top P       | Rock Concert         | Theater Performance  | Rock Concert         | Theater Performance  |
| 0.5 | 1                 | 97.34%               | 2.31%                | 91.95%               | 7.67%                |
| 1 | 1                  | 97.35%               | 2.30%                | 91.95%               | 7.67%                |
| 1.5 | 1                 | 97.35%               | 2.31%                | 91.94%               | 7.67%                |
| 1 | 0.5                 | 97.34%               | 2.31%                | 92.55%               | 7.07%                |

Model: gpt-3.5-turbo-instruct

Changing the parameters `Temperature` and `Top P` does not change the probabilities. But changing the amount of sunk costs from `$90` to `$250` slightly increases the probability of choosing the theater performance by about 5 percentage points.

In conclusion, although the model (gpt-3.5-turbo-instruct) isn't significantly affected by sunk cost bias, a slight impact is still observed.

---

### Using Chat Completion API with gpt-4-1106-preview ("GPT-4 Turbo")

Result: The model did not choose the theater performance once. Therefore GPT-4 Turbo does not seem to be subject to the sunk cost bias.

In [4]:
def sunk_cost_experiment(model, prompt, instruction, n, temperature=1, max_tokens=2):
    answers = []
    for _ in range(n):
        response = client.chat.completions.create(
            model=model,  
            messages=[
                {"role": "system", "content": instruction},

                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature
        )

        # Store the answer in the list
        answer = response.choices[0].message.content
        answers.append(answer.strip())

        # Add delay before the next API call
        time.sleep(0.1)  
     
    return answers

In [1]:
# gpt-4-1106-preview (GPT-4 Turbo) is more powerful than GPT-4 and offered at a lower price
model = 'gpt-4-1106-preview'

instruction = 'Complete the answer with two words.'

prompt = """
    Assume that you have spent $250 for a ticket to a theater performance. 
    Several weeks later you buy a $30 ticket to a rock concert. You think you 
    will enjoy the rock concert more than the theater performance. As you are 
    putting your just-purchased rock concert ticket in your wallet, you notice 
    that both events are scheduled for the same evening. The tickets are 
    non-transferable, nor can they be exchanged. You can use only one of the 
    tickets and not the other. Which ticket will you use?

    Answer: I would use the ticket to the
    """

In [51]:
%%time

answers = sunk_cost_experiment(model, prompt, instruction, n=100)

Wall time: 53 s


In [53]:
answers.count('rock concert') / len(answers)

1.0

In [54]:
answers.count('theater performance') / len(answers)

0.0

---

### Using Chat Completion API with gpt-3.5-turbo-1106

Result: Also not subject to the sunk cost bias.

In [16]:
model = 'gpt-3.5-turbo-1106'

In [19]:
%%time

answers = sunk_cost_experiment(model, prompt, instruction, n=100)

Wall time: 15min 9s


In [ ]:
# Note: Why does it take so long (~15min)?

In [20]:
answers.count('rock concert') / len(answers)

0.95

In [21]:
answers.count('theater performance') / len(answers)

0.0

---

### Testing Completion API (not finished)

In [23]:
response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt="Write a tagline for an ice cream shop."
)

In [34]:
response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
    prompt="""
    Assume that you have spent $250 for a ticket to a theater performance. 
    Several weeks later you buy a $30 ticket to a rock concert. You think 
    you will enjoy the rock concert more than the theater performance. As 
    you are putting your just-purchased rock concert ticket in your wallet, 
    you notice that both events are scheduled for the same evening. The 
    tickets are non-transferable, nor can they be exchanged. You can use 
    only one of the tickets and not the other. Which ticket will you use?

    \n\nAnswer: I would use the ticket to the
    """,
  temperature=1,
  max_tokens=2,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  logprobs=3
)

In [35]:
response

Completion(id='cmpl-8KnX1HHywTdumI14MwH3FYGdWlvIQ', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=Logprobs(text_offset=[574, 579], token_logprobs=[-0.27882972, -0.00016325322], tokens=[' rock', ' concert'], top_logprobs=[{' rock': -0.27882972, ' theater': -1.8193265, 'rock': -2.8857024}, {' concert': -0.00016325322, '\n': -9.928477, ' ': -10.569514}]), text=' rock concert')], created=1699967651, model='gpt-3.5-turbo-instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=2, prompt_tokens=128, total_tokens=130))